In [ ]:
!nvidia-smi

Fri Mar 17 01:42:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.20.1
!pip install sentencepiece==0.1.97
!pip install tokenizers==0.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 75.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip -qqq install sentence-transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.4 MB/s eta 0:00:00


# Check
- exp_noをかえたか？: ok

In [ ]:
# =========================================================================================
# Libraries
# =========================================================================================
import os
import gc
import time
import math
import random
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_cosine_schedule_with_warmup, DataCollatorWithPadding#
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GroupKFold

#if torch.cuda.is_available():
#    import cupy as cp
#    from cuml.metrics import pairwise_distances
#    from cuml.neighbors import NearestNeighbors

%env TOKENIZERS_PARALLELISM=true
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# =========================================================================================
# Config
# =========================================================================================
class CFG: 
    exp_no = "Retriever054" # 必ず確認する＆変更する！
    # ↓必ず確認する＆変更する！
    model = "sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1"
    model_name = Path(model).name
    tokenizer = AutoTokenizer.from_pretrained(model)
    epochs = 20
    batch_size = 128
    debug = False
    #calc_score_before_training = False

    # Load Paths(exp): 必ず確認し、必要な場合変更する。
    topics_retriever_training = f"/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/01_for_retriever/Retriever037/topics_with_fold.pkl"

    # Load Paths(Common): 基本変えないので変更不要: 036で変えたので注意
    # competition_data
    compdata_topics = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/topics.csv"
    compdata_content = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/content.csv"
    compdata_correlations = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/correlations.csv"
    compdata_sample_sub = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/sample_submission.csv"
    # contents_for_retriever: 036から変更
    contents_processed_for_ret = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/01_for_retriever/contents-for-retriever_v4/content_preprocessed_for_retriever.pkl"

    # Save Paths: 動的に変わるので変更不要
    model_save_path = f"/content/drive/MyDrive/Colab_Files/kaggle/lecr/models/{exp_no}/model_trained"
    predictions_save_path = f"/content/drive/MyDrive/Colab_Files/kaggle/lecr/models/{exp_no}/predictions"


# =========================================================================================
# Seed everything for deterministic results
# =========================================================================================
def seed_everything(cfg):
    random.seed(cfg.seed)
    os.environ['PYTHONHASHSEED'] = str(cfg.seed)
    np.random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)
    torch.cuda.manual_seed(cfg.seed)
    torch.backends.cudnn.deterministic = True
    
# =========================================================================================
# Utils
# =========================================================================================
import pickle

def show_df(df:pd.DataFrame, rows_=5):
    print(df.shape)
    return display(df.head(rows_))

def pickle_dump(obj_, filename):
    with open(f'{filename}', 'wb') as f:
        pickle.dump(obj_, f)
        
def pickle_load(path_):
    with open(f'{path_}', 'rb') as f:
        obj_ = pickle.load(f)
    return obj_

env: TOKENIZERS_PARALLELISM=true


Downloading:   0%|          | 0.00/556 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from datasets import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

In [ ]:
topics_train = pickle_load(CFG.topics_retriever_training)
content = pickle_load(CFG.contents_processed_for_ret)
correlations = pd.read_csv(CFG.compdata_correlations)
topics_train = topics_train.add_prefix('topic_')
content = content.add_prefix('content_')

In [ ]:
correlations["content_id"] = correlations["content_ids"].str.split(" ")
corr = correlations.explode("content_id").drop(columns=["content_ids"])
corr.head()

,topic_id,content_id
0,t_00004da3a1b2,c_1108dd0c7a5d
0,t_00004da3a1b2,c_376c5a8eb028
0,t_00004da3a1b2,c_5bc0e1e2cba0
0,t_00004da3a1b2,c_76231f9d0b5e
1,t_00068291e9a4,c_639ea2ef9c95


In [ ]:
corr = corr.merge(topics_train[['topic_id', 'topic_all_text', 'topic_fold']], how="left", on="topic_id")
corr = corr.merge(content[['content_id', 'content_all_text']], how="left", on="content_id")
corr.head()

,topic_id,content_id,topic_all_text,topic_fold,content_all_text
0,t_00004da3a1b2,c_1108dd0c7a5d,NaN,NaN,[kind=video][SEP][language=bg][SEP]Молив като ...
1,t_00004da3a1b2,c_376c5a8eb028,NaN,NaN,[kind=video][SEP][language=bg][SEP]Да чуем про...
2,t_00004da3a1b2,c_5bc0e1e2cba0,NaN,NaN,[kind=video][SEP][language=bg][SEP]Променлив р...
3,t_00004da3a1b2,c_76231f9d0b5e,NaN,NaN,[kind=video][SEP][language=bg][SEP]Последовате...
4,t_00068291e9a4,c_639ea2ef9c95,NaN,NaN,[kind=exercise][SEP][language=pt][SEP]Dados e ...


In [ ]:
corr = corr.rename(columns={'topic_fold':'fold'})

In [ ]:
corr_tr = corr[corr['fold'] != 2]

In [ ]:
corr_tr["set"] = corr_tr[["topic_all_text", "content_all_text"]].values.tolist()
train_df = pd.DataFrame(corr_tr["set"])
dataset = Dataset.from_pandas(train_df)
train_examples = []
train_data = dataset["set"]
n_examples = dataset.num_rows

for i in range(n_examples):
    example = train_data[i]
    if example[0] == None: #remove None
        continue        
    train_examples.append(InputExample(texts=[str(example[0]), str(example[1])]))

model = SentenceTransformer(CFG.model)

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CFG.batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
num_epochs = CFG.epochs
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          save_best_model = True,
          output_path='./model_trained_tmp',
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/131 [00:00<?, ?it/s]

In [ ]:
model.save(f"{CFG.model_save_path}")

In [ ]:
from google.colab import runtime
runtime.unassign()